In [8]:
from IPython.display import display, JSON
#display(result)

In [20]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate('./nugunaaiot-maeng-1004a11a5af7.json')

app = firebase_admin.initialize_app(cred)

db = firestore.client()


ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [21]:
# 데이터 추가
# 데이터 스냅샷(가져오기) get, 참조는  ref
doc_ref = db.collection('customer').document('c1')
doc_ref.set({
    'c_id': 'c1',
    'name': '최원칠',
    'total_amount':500,
    'rat': 'silver'
})


update_time {
  seconds: 1670658351
  nanos: 785326000
}

In [22]:
# 데이터 id로 읽기
doc_ref = db.collection('customer').document('c1')

doc = doc_ref.get()

print(doc)

In [23]:
if doc.exists:
    print(f'Document data: {doc.to_dict()}')
else:
    print(u'No such document!')

Document data: {'c_id': 'c1', 'rat': 'silver', 'total_amount': 500, 'name': '최원칠'}


In [24]:
# 데이터 전체 읽기
users_ref = db.collection('customer')
docs = users_ref.stream()

print(docs)

<generator object Query.stream at 0x000001AD18A45690>


In [25]:
for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

1c => {'name': 'hera'}
c1 => {'c_id': 'c1', 'rat': 'silver', 'total_amount': 500, 'name': '최원칠'}


In [26]:
# 데이터 쿼리
cities_ref = db.collection('customer')
q = cities_ref.where('total_amount', '<=', 1000).stream() # 동일 컬럼에서만 중복 쿼리 가능
for doc in q:
    print(f'{doc.id} => {doc.to_dict()}')

c1 => {'c_id': 'c1', 'rat': 'silver', 'total_amount': 500, 'name': '최원칠'}


In [27]:
# 데이터 업데이트
doc_ref = db.collection('customer').document('c1')

doc_ref.update({'account_num': 1,
                'accounts':{'11a':{'a_id':'11a', 'c_id':'c1', 'amount':100}}
               })

doc_ref = db.collection('customer').document('c1')

doc = doc_ref.get()
if doc.exists:
    print(f'Document data: {doc.to_dict()}')
else:
    print(u'No such document!')


Document data: {'name': '최원칠', 'c_id': 'c1', 'rat': 'silver', 'total_amount': 500, 'account_num': 1, 'accounts': {'11a': {'c_id': 'c1', 'amount': 100, 'a_id': '11a'}}}


In [28]:
# 데이터 업데이트
doc_ref = db.collection('customer').document('c1')

# 특수 필드 사용
# 특정 맵의 필드를 추가하기위해 < 맵.필드 : 값 > 사용
doc_ref.update({'account_num': firestore.Increment(1),# +=
                'total_amount' : firestore.Increment(400),
                'edit_time': firestore.SERVER_TIMESTAMP, # 숨겨진 칼럼으로 fb기능을 사용가능
                'accounts.12a':{'a_id':'12a', 'c_id':'c1', 'amount':400} # 콜렉션.필드id 를 키값으로 사용가능
               })

doc_ref = db.collection('customer').document('c1')

doc = doc_ref.get()
if doc.exists:
    print(f'Document data: {doc.to_dict()}')
else:
    print(u'No such document!')

Document data: {'accounts': {'11a': {'c_id': 'c1', 'amount': 100, 'a_id': '11a'}, '12a': {'c_id': 'c1', 'amount': 400, 'a_id': '12a'}}, 'account_num': 2, 'c_id': 'c1', 'edit_time': DatetimeWithNanoseconds(2022, 12, 10, 7, 47, 47, 272000, tzinfo=datetime.timezone.utc), 'rat': 'silver', 'total_amount': 900, 'name': '최원칠'}


In [11]:
# py파일 또는 ipynb 파일 새로 생성하여 진행
# 파이어스토어 db 인스턴스 생성
# bank4.json 파일내의 데이터를 파이어스토어 customer 컬렉션에 고객 아이디를 아이디로 가지는 문서들로 추가
# {아이디: 5c, 이름: 본인이름, 계좌개수:0, 총금액:0, 등급:normal, 계좌:비어있는 딕셔너리} 고객 customer 컬렉션에 추가
# 5c 고객에 임의의 계좌 추가 (고객의 계좌, 총금액, 계좌개수, 등급, 수정시간 업데이트되어야함)
# 계좌개수, 총금액은 firestore.Increment() 사용

import json

with open('bank4.json','r') as f:
    data = json.load(f)


In [12]:
display(data)

{'1c': {'c_id': '1c',
  'name': '최원칠',
  'account_num': 2,
  'total_amount': 500,
  'rat': 'silver',
  'account': {'11a': {'a_id': '11a',
    'password': '11p',
    'c_id': '1c',
    'amount': 100},
   '12a': {'a_id': '12a', 'password': '12p', 'c_id': '1c', 'amount': 400}}},
 '2c': {'c_id': '2c',
  'name': '이서혁',
  'account_num': 1,
  'total_amount': 2000,
  'rat': 'gold',
  'account': {'21a': {'a_id': '21a',
    'password': '21p',
    'c_id': '2c',
    'amount': 2000}}},
 '3c': {'c_id': '3c',
  'name': 'aiot',
  'account_num': 3,
  'total_amount': 50000,
  'rat': 'vip',
  'account': {'31a': {'a_id': '31a',
    'password': '31p',
    'c_id': '3c',
    'amount': 10000},
   '32a': {'a_id': '32a', 'password': '32p', 'c_id': '3c', 'amount': 20000},
   '33a': {'a_id': '33a', 'password': '33p', 'c_id': '3c', 'amount': 20000}}},
 '4c': {'c_id': '4c',
  'name': '새싹',
  'account_num': 4,
  'total_amount': 200000,
  'rat': 'vvip',
  'account': {'41a': {'a_id': '41a',
    'password': '41p',
    '

In [19]:
# 트랜젝션

transaction = db.transaction()
doc_ref = db.collection('customer').document('c1')

@firestore.transactional
def update_in_transaction(transaction, doc_ref):
    snapshot = doc_ref.get(transaction=transaction)
    total_amount = snapshot.get('total_amount')
    ntotal_amount = total_amount + 1000
    
    transaction.update(doc_ref, {
            'total_amount': ntotal_amount
        })
    
    return ntotal_amount

result = update_in_transaction(transaction, doc_ref)
result

1500